In [21]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [53]:
test = pd.read_csv("C:/Users/adith/Desktop/Datasets/Linear_test.csv")
train = pd.read_csv("C:/Users/adith/Desktop/Datasets/Linear_train.csv")
x_train_scaled = (train - train.mean()) / (train.std()) 
x_test_scaled = (test - test.mean()) / (test.std())
x_train_scaled.head()

,Unnamed: 0,A,B,C,D,E,F,G,H,I,...,L,M,N,O,P,Q,R,S,T,label
0,-1.731999,-0.347991,0.220312,0.285110,1.133715,0.678330,-0.610903,0.898280,1.199218,-0.424613,...,0.138291,1.028725,-0.863910,0.423651,-1.184218,1.284498,-0.155822,1.005271,1.008755,-0.248061
1,-1.731930,-0.611140,0.199847,0.427126,-1.254843,0.220549,-0.617274,0.443115,-1.502330,1.151147,...,1.707571,-0.056831,0.558906,1.497778,-0.253123,1.572980,1.294387,0.446828,-1.123392,0.982331
2,-1.731860,-0.982550,-0.677569,-1.233452,-0.307475,-1.573969,0.070213,-0.010727,0.623922,-1.172975,...,-0.930919,-0.252847,0.969013,0.928881,0.668976,-1.243750,-0.206644,0.520149,-1.184463,0.215179
3,-1.731791,-1.403101,0.385949,-1.001564,0.080227,-1.201313,1.310732,0.820607,-1.678859,-0.554849,...,-0.242377,-0.441135,1.663925,1.291192,1.064841,0.763239,0.565756,1.432712,1.162713,2.434678
4,-1.731722,-1.266513,1.216147,-1.131821,-0.043858,-0.483668,0.241872,1.520124,1.150941,-1.390864,...,-0.839817,0.369288,-1.631806,-0.274098,-0.277774,1.352054,-1.712420,-1.124743,-1.253185,-1.416104


In [57]:
y = x_train_scaled["label"]
y_tests = x_test_scaled["label"]
print(y.shape)
x_train_scaled.drop(x_train_scaled.columns[[0, 21]], axis=1 , inplace=True)
x_test_scaled.drop(x_test_scaled.columns[[0, 21]], axis=1 , inplace=True)

(50000,)


In [58]:
X_train = x_train_scaled.values
X_train = X_train.T
print(X_train.shape)

Y_train = np.array([y])
Y_train = Y_train.reshape(1,50000)
print(Y_train.shape)
X_test = x_test_scaled.values
X_test = X_test.T
Y_test = np.array([y_tests])
Y_test = Y_test.reshape(1,10000)
print(Y_test.shape)

(20, 50000)
(1, 50000)
(1, 10000)


In [59]:
def tanh(x):
    return np.tanh(x)

def relu(x):
    return np.maximum(x, 0)

def softmax(x):
    expX = np.exp(x)
    return expX/np.sum(expX, axis = 0)

In [60]:
def derivative_tanh(x):
    return (1 - np.power(np.tanh(x), 2))

def derivative_relu(x):
    return np.array(x > 0, dtype = np.float32)

In [61]:
def initialize_parameters(n_x, n_h, n_y):
    w1 = np.random.randn(n_h, n_x)
    b1 = np.zeros((n_h, 1))
    
    w2 = np.random.randn(n_y, n_h)
    b2 = np.zeros((n_y, 1))
    
    parameters = {
        "w1" : w1,
        "b1" : b1,
        "w2" : w2,
        "b2" : b2
    }
    
    return parameters

In [62]:
def forward_propagation(x, parameters):
    
    w1 = parameters['w1']
    b1 = parameters['b1']
    w2 = parameters['w2']
    b2 = parameters['b2']
    
    z1 = np.dot(w1, x) + b1
    a1 = z1
    
    z2 = np.dot(w2, a1) + b2
    a2 = z2
    
    forward_cache = {
        "z1" : z1,
        "a1" : a1,
        "z2" : z2,
        "a2" : a2
    }
    
    return forward_cache

In [63]:
def cost_function(a2, y):
    m = y.shape[1]
    
    cost = (1/2*m)*np.sum(np.square(y-a2))
    
    #cost = -(1/m)*np.sum(np.sum(y*np.log(a2, 0), 1))
    
    return cost

In [64]:
def backward_prop(x, y, parameters, forward_cache):
    
    w1 = parameters['w1']
    b1 = parameters['b1']
    w2 = parameters['w2']
    b2 = parameters['b2']
    
    a1 = forward_cache['a1']
    a2 = forward_cache['a2']
    
    m = x.shape[1]
    
    dz2 = (a2 - y)
    dw2 = (1/m)*np.dot(dz2, a1.T)
    db2 = (1/m)*np.sum(dz2, axis = 1, keepdims = True)
    
    dz1 = (1/m)*np.dot(w2.T, dz2)*derivative_relu(a1)
    dw1 = (1/m)*np.dot(dz1, x.T)
    db1 = (1/m)*np.sum(dz1, axis = 1, keepdims = True)
    
    gradients = {
        "dw1" : dw1,
        "db1" : db1,
        "dw2" : dw2,
        "db2" : db2
    }
    
    return gradients

In [65]:
def update_parameters(parameters, gradients, learning_rate):
    
    w1 = parameters['w1']
    b1 = parameters['b1']
    w2 = parameters['w2']
    b2 = parameters['b2']
    
    dw1 = gradients['dw1']
    db1 = gradients['db1']
    dw2 = gradients['dw2']
    db2 = gradients['db2']
    
    w1 = w1 - learning_rate*dw1
    b1 = b1 - learning_rate*db1
    w2 = w2 - learning_rate*dw2
    b2 = b2 - learning_rate*db2
    
    parameters = {
        "w1" : w1,
        "b1" : b1,
        "w2" : w2,
        "b2" : b2
    }
    
    return parameters


In [66]:
def model(x, y, n_h, learning_rate, iterations):
    
    n_x = x.shape[0]
    n_y = y.shape[0]
    
    cost_list = []
    
    parameters = initialize_parameters(n_x, n_h, n_y)
    
    for i in range(iterations):
        
        forward_cache = forward_propagation(x, parameters)
        
        cost = cost_function(forward_cache['a2'], y)
        
        gradients = backward_prop(x, y, parameters, forward_cache)
        
        parameters = update_parameters(parameters, gradients, learning_rate)
        
        cost_list.append(cost)
        
        if(i%(iterations/10) == 0):
            print("Cost after", i, "iterations is :", cost)
        
    return parameters, cost_list

In [67]:
iterations = 1000
n_h = 1
learning_rate = 0.03
Parameters, Cost_list = model(X_train, Y_train, n_h = n_h, learning_rate = learning_rate, iterations = iterations)

Cost after 0 iterations is : 9279272176.83368
Cost after 100 iterations is : 1237902130.2107024
Cost after 200 iterations is : 1237902100.6133277
Cost after 300 iterations is : 1237902071.0158818
Cost after 400 iterations is : 1237902041.4183636
Cost after 500 iterations is : 1237902011.8207736
Cost after 600 iterations is : 1237901982.223112
Cost after 700 iterations is : 1237901952.6253786
Cost after 800 iterations is : 1237901923.0275733
Cost after 900 iterations is : 1237901893.429696


In [68]:
def predict(X_test, parameters):
    forward_cache = forward_propagation(X_test, parameters)
    return forward_cache["a2"]

In [69]:
y_pred = predict(X_test, Parameters)

In [70]:
y_pred.shape

(1, 10000)

In [71]:
def accuracy(Y_test, y_pred):
    return np.sqrt((np.square(Y_test - y_pred)).mean())
print("Rmse error on test dataset = ", accuracy(Y_test, y_pred))

Rmse error on test dataset =  0.99367818953251
